In [147]:
import pandas as pd
from sklearn.externals import joblib
import json
import seaborn as sns
import numpy as np
from sklearn.svm import SVR

In [148]:
svr = SVR(gamma="auto",kernel="linear", C=10, degree=4)

In [62]:
df = pd.read_csv("data/recipe_0_out_balec_reg.csv", index_col=0)

In [202]:
val = pd.read_csv("data/recipe_0_val_reg.csv", index_col=0)

In [204]:
df=df.dropna()

In [205]:
df.head()

,line_dd,moisture_x,bulk_density,sifter_speed_nominal_pct,steam_pres_avg_10,steam_pres_min_10,steam_pres_max_10,prod_temp_jetcooker_avg_10,prod_temp_jetcooker_min_10,prod_temp_jetcooker_max_10,...,temp_out,water_pct,water_correction,fat_pct,particles_grp1,particles_grp2,particles_grp3,moisture_y,usage_pct,elems
0,DD08,3.97,230,0.75,7.997975,7.995008,8.002723,108.432023,105.379082,110.847687,...,118,0.66,-60,0.060604,0.537341,0.210579,0.233674,11.4533,1,2
1,DD08,4.05,200,0.75,8.003845,7.999421,8.010713,105.958855,103.057098,108.308693,...,118,0.66,-60,0.060604,0.537341,0.210579,0.233674,11.4533,1,2
2,DD08,4.00,200,0.75,8.157459,8.124105,8.175771,107.825485,105.227173,110.652374,...,118,0.66,-60,0.060604,0.537341,0.210579,0.233674,11.4533,1,2
3,DD08,3.87,200,0.75,8.190472,8.181672,8.207918,106.623262,103.968529,109.089920,...,118,0.66,-60,0.060604,0.537341,0.210579,0.233674,11.4533,1,2
4,DD08,4.21,200,0.75,8.228980,8.216983,8.236519,116.805276,116.424789,117.032410,...,118,0.66,-60,0.060604,0.537341,0.210579,0.233674,11.4533,1,2


In [206]:
df_train = df.iloc[:250]
df_test = df.iloc[250:]

## Moisture models

In [232]:
cols_to_fit = ['dd_speed', 'particles_grp1',
              'particles_grp2', 'particles_grp3', 'elems', 'water_correction']

## Mean model

In [233]:
(((df['moisture_x'] - df['moisture_x'].mean())**2).mean())

0.4183451233333334

#### from xgboost import XGBRegressor

In [234]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score
import sklearn
from sklearn.svm import SVC
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [235]:
xgb = XGBRegressor(booster='gblinear')

In [236]:
rf = RandomForestRegressor(n_estimators=50, max_depth=3, min_samples_split=3,
                           min_samples_leaf=2)

In [237]:
rf.fit(df_train[cols_to_fit], df_train['moisture_x'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [238]:
xgb.fit(df_train[cols_to_fit],df_train['moisture_x'])

XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [239]:
(mean_squared_error(df_test['moisture_x'], rf.predict(df_test[cols_to_fit])))

0.24320800401456316

In [240]:
lr = LinearRegression()

In [241]:
rf.fit(df[cols_to_fit], df['moisture_x'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [242]:
val['moisture_pred'] = rf.predict(val[cols_to_fit])

In [172]:
joblib.dump(rf, "recepta_0_model_moisture.h5")

['recepta_0_model_moisture.h5']

In [173]:
with open("recepta_0_cols_moisture.json", "w") as f:
    json.dump(cols_to_fit, f)

In [174]:
from sklearn.feature_selection import RFE

## Bulk density

In [243]:
(((df['bulk_density'] - df['bulk_density'].mean())**2).mean())

193.80795524691356

In [244]:
rf = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=10,
                           min_samples_leaf=2)

In [245]:
rf.fit(df_train[cols_to_fit], df_train['bulk_density'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [246]:
val['bulk_density_pred'] = rf.predict(val[cols_to_fit])

In [247]:
val.to_csv("recept_0_reg_pred.csv")

In [142]:
joblib.dump(rf, "recepta_0_model_bulk_density.h5")

['recepta_0_model_bulk_density.h5']

In [143]:
with open("recepta_0_cols_bulk_density.json", "w") as f:
    json.dump(cols_to_fit, f)

In [193]:
(mean_squared_error(df_test['bulk_density'], rf.predict(df_test[cols_to_fit])))

321.54019218139354

In [194]:
(mean_squared_error(df_test['bulk_density'], np.ones_like(df_test['bulk_density']) * 
                                                          df_test['bulk_density'].mean()))

196.60330578512404